<a href="https://colab.research.google.com/github/anderoak/projeto_consultor_ai/blob/main/Consultor_AI_de_Documentos_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Consultor AI de Documentos PDF
Este notebook demonstra um projeto que combina a API Gemini com embeddings para criar um chatbot capaz de responder perguntas sobre o conteúdo de documentos e buscar informações relevantes em uma base de documentos PDF.

Copyright © 2023 [Anderson Carvalho] - GNU GENERAL PUBLIC LICENSE

##Instruções para Utilização

-  Adicione a sua chave de API do Google AI no Secrets, na barra lateral esquerda do Colab, definindo com o nome: GEMINI_API_KEY.
-  Crie uma pasta chamada PDF no ambiente Colab. Você pode fazer clicando no ícone arquivos na lateral esquerda do Colab, depois clicando com o botão direito no espaço em branco e selecionando "Nova pasta".
- Depois faça upload dos seus arquivos PDF e arraste-os para dentro da pasta PDF que criou. Leve em conta que a API do Gemini pode limitar de quantidade de dados carregados dos PDF se você está usando a versão gratuita da API, então use apenas um ou dois arquivos pequenos (deixei 2 arquivos que usei para testar no [repositório do GitHub](https://github.com/anderoak/projeto_consultor_ai), na pasta "PDFs para teste").
- Garanta que os arquivos na pasta sejam realmente PDFs. Atente que esse projeto assume que os PDFs contêm texto extraível, portanto não irá funcionar com PDFs que são apenas imagens digitalizadas, pois exigiria técnicas de OCR (Optical Character Recognition) para extrair o texto.
-  Execute o código célula por célula.
-  Interaja com o chatbot fazendo perguntas sobre leis.
-  Digite "sair" para encerrar o chat.

Observação: Este é um exemplo básico genérico, mas que talvez possa se aprimorado adicionando mais funcionalidades, como especialização por área e uma interface web.

##1. Instalação de Bibliotecas

In [3]:
# Instala as bibliotecas necessárias
!pip install -q -U google-generativeai pypdf2

# Importa as bibliotecas
import google.generativeai as genai
from google.colab import userdata
import numpy as np
import PyPDF2

# Lê a API Key do Secrets do Colab
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Funções para processar PDFs

In [4]:
def extrair_texto_pdf(caminho_arquivo):
  """
  Extrai o texto de um arquivo PDF.
  """
  with open(caminho_arquivo, 'rb') as arquivo_pdf:
    leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)
    texto = ""
    for num_pagina in range(len(leitor_pdf.pages)):
      pagina = leitor_pdf.pages[num_pagina]
      texto += pagina.extract_text()
    return texto

def carregar_documentos_pdf(pasta_colab='./PDF/'):
  """
  Carrega os arquivos PDF de uma pasta no Colab e extrai o texto.
  """
  import os
  arquivos_pdf = [os.path.join(pasta_colab, arquivo)
                   for arquivo in os.listdir(pasta_colab)
                   if arquivo.endswith('.pdf')]

  # Extrai o texto de cada arquivo PDF
  documentos = [extrair_texto_pdf(arquivo) for arquivo in arquivos_pdf]
  return documentos

##2. Configuração do Modelo Gemini

In [5]:
# Configura os parâmetros do modelo Gemini
generation_config = {
    "candidate_count": 1,  # Número de respostas a serem geradas
    "temperature": 0.5,   # Controla a criatividade das respostas
}

# Configura os parâmetros de segurança
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

# Inicializa o modelo Gemini
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Define o modelo de embedding
embedding_model = "models/embedding-001"

##3. Carregamento da Base de Dados e Embeddings

In [11]:
# Carrega os documentos da pasta do Colab
documentos = carregar_documentos_pdf()

# Gera embeddings para os documentos usando Gemini
embeddings_documentos = []
for documento in documentos:
  embedding = genai.embed_content(model=embedding_model,
                                  content=documento,
                                  task_type="RETRIEVAL_DOCUMENT")["embedding"]
  embeddings_documentos.append(embedding)

## 4. Função de Busca por Similaridade

In [7]:
def buscar_documento(pergunta, embeddings_documentos, documentos):
  """
  Busca o documento mais similar à pergunta usando embeddings.
  """
  # Gera o embedding da pergunta usando Gemini
  embedding_pergunta = genai.embed_content(model=embedding_model,
                                           content=pergunta,
                                           task_type="RETRIEVAL_QUERY")["embedding"]

  # Calcula a similaridade cosseno entre a pergunta e os documentos
  similaridades = np.dot(np.stack(embeddings_documentos), embedding_pergunta)

  # Encontra o índice do documento mais similar
  indice_documento_similar = np.argmax(similaridades)

  # Retorna o documento mais similar
  return documentos[indice_documento_similar]

## 5. Interação com o Chatbot

In [ ]:
# Inicia o chat
chat = model.start_chat(history=[])

# Loop de interação
while True:
  # Obtém a pergunta do usuário
  pergunta = input("Pergunta: ")

  # Busca o documento mais similar
  documento_similar = buscar_documento(pergunta, embeddings_documentos, documentos)

  # Envia a pergunta e o documento para o Gemini
  resposta = chat.send_message(f"Pergunta: {pergunta}\nContexto: {documento_similar}")

  # Exibe a resposta do Gemini
  print(f"Resposta: {resposta.text}")

  # Condição de parada
  if pergunta.lower() == 'sair':
    break